# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 05:59:45) [MSC v.1929 64 bit (AMD64)]
Pandas 1.4.3
Scikit-Learn 1.1.1
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import os

In [3]:
classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(8, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(16, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Flattening
classifier.add(Flatten())

#Full connection
classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dropout(0.30))
classifier.add(Dense(units = 1, activation= 'sigmoid'))

Let's double check the model description:

In [4]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 21904)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                700960    
_________________________________________________________________
dropout (Dropout)            (None, 32)                0

In [5]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.00001)
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_dir = '../../dataset/train'
parth_dir_test = '../../dataset/test'

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    shuffle=True,
    subset='training',
    batch_size = 64,
    class_mode = 'binary'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    subset='validation',
    batch_size = 64,
    class_mode = 'binary'
)


Found 2202 images belonging to 2 classes.
Found 550 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'nonviolence': 0, 'violence': 1}

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=30)
mc = ModelCheckpoint('best_cnn.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20)

# Create a TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
classifier.fit(train_generator,
                         epochs = 200,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_generator
                         )

Epoch 1/200
35/35 [==============================] - 14s 273ms/step - loss: 0.6864 - accuracy: 0.5372 - val_loss: 0.6773 - val_accuracy: 0.6236
Epoch 2/200
35/35 [==============================] - 7s 204ms/step - loss: 0.6771 - accuracy: 0.5904 - val_loss: 0.6680 - val_accuracy: 0.7073
Epoch 3/200
35/35 [==============================] - 7s 214ms/step - loss: 0.6700 - accuracy: 0.6167 - val_loss: 0.6573 - val_accuracy: 0.6964
Epoch 4/200
35/35 [==============================] - 8s 240ms/step - loss: 0.6618 - accuracy: 0.6535 - val_loss: 0.6458 - val_accuracy: 0.7600
Epoch 5/200
35/35 [==============================] - 7s 207ms/step - loss: 0.6502 - accuracy: 0.6735 - val_loss: 0.6406 - val_accuracy: 0.6436
Epoch 6/200
35/35 [==============================] - 7s 206ms/step - loss: 0.6406 - accuracy: 0.6857 - val_loss: 0.6257 - val_accuracy: 0.7436
Epoch 7/200
35/35 [==============================] - 7s 212ms/step - loss: 0.6332 - accuracy: 0.7016 - val_loss: 0.6129 - val_accuracy: 0.787

Epoch 58/200
35/35 [==============================] - 7s 213ms/step - loss: 0.4386 - accuracy: 0.8215 - val_loss: 0.4155 - val_accuracy: 0.8545
Epoch 59/200
35/35 [==============================] - 7s 208ms/step - loss: 0.4419 - accuracy: 0.8174 - val_loss: 0.4126 - val_accuracy: 0.8600
Epoch 60/200
35/35 [==============================] - 8s 231ms/step - loss: 0.4456 - accuracy: 0.8129 - val_loss: 0.4185 - val_accuracy: 0.8509
Epoch 61/200
35/35 [==============================] - 8s 226ms/step - loss: 0.4406 - accuracy: 0.8179 - val_loss: 0.4114 - val_accuracy: 0.8600
Epoch 62/200
35/35 [==============================] - 8s 216ms/step - loss: 0.4424 - accuracy: 0.8311 - val_loss: 0.4106 - val_accuracy: 0.8600
Epoch 63/200
35/35 [==============================] - 8s 215ms/step - loss: 0.4386 - accuracy: 0.8179 - val_loss: 0.4107 - val_accuracy: 0.8600
Epoch 64/200
35/35 [==============================] - 8s 218ms/step - loss: 0.4396 - accuracy: 0.8256 - val_loss: 0.4106 - val_accuracy:

Epoch 115/200
35/35 [==============================] - 7s 211ms/step - loss: 0.3831 - accuracy: 0.8565 - val_loss: 0.3706 - val_accuracy: 0.8691
Epoch 116/200
35/35 [==============================] - 7s 209ms/step - loss: 0.3818 - accuracy: 0.8506 - val_loss: 0.3713 - val_accuracy: 0.8745
Epoch 117/200
35/35 [==============================] - 7s 211ms/step - loss: 0.3888 - accuracy: 0.8479 - val_loss: 0.3715 - val_accuracy: 0.8655
Epoch 118/200
35/35 [==============================] - 7s 213ms/step - loss: 0.3741 - accuracy: 0.8565 - val_loss: 0.3688 - val_accuracy: 0.8709
Epoch 119/200
35/35 [==============================] - 7s 206ms/step - loss: 0.3758 - accuracy: 0.8547 - val_loss: 0.3684 - val_accuracy: 0.8709
Epoch 120/200
35/35 [==============================] - 8s 216ms/step - loss: 0.3765 - accuracy: 0.8520 - val_loss: 0.3674 - val_accuracy: 0.8782
Epoch 121/200
35/35 [==============================] - 7s 207ms/step - loss: 0.3784 - accuracy: 0.8597 - val_loss: 0.3681 - val_ac

35/35 [==============================] - 7s 212ms/step - loss: 0.3373 - accuracy: 0.8728 - val_loss: 0.3337 - val_accuracy: 0.8855
Epoch 172/200
35/35 [==============================] - 7s 212ms/step - loss: 0.3313 - accuracy: 0.8742 - val_loss: 0.3387 - val_accuracy: 0.8673
Epoch 173/200
35/35 [==============================] - 7s 211ms/step - loss: 0.3320 - accuracy: 0.8751 - val_loss: 0.3384 - val_accuracy: 0.8655
Epoch 174/200
35/35 [==============================] - 7s 213ms/step - loss: 0.3361 - accuracy: 0.8728 - val_loss: 0.3334 - val_accuracy: 0.8782
Epoch 175/200
35/35 [==============================] - 8s 217ms/step - loss: 0.3352 - accuracy: 0.8819 - val_loss: 0.3348 - val_accuracy: 0.8727
Epoch 176/200
35/35 [==============================] - 8s 216ms/step - loss: 0.3282 - accuracy: 0.8774 - val_loss: 0.3303 - val_accuracy: 0.8873
Epoch 177/200
35/35 [==============================] - 8s 221ms/step - loss: 0.3305 - accuracy: 0.8797 - val_loss: 0.3305 - val_accuracy: 0.8873

In [10]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

#https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d

In [13]:
%tensorboard --logdir logs/fit

# Part 3 test the CNN

In [14]:
# Load Model
classifier.load_weights('best_cnn.h5')

In [15]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 21904)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                700960    
_________________________________________________________________
dropout (Dropout)            (None, 32)                0

In [16]:
test_set = datagen.flow_from_directory(parth_dir_test,
                                        target_size= (150,150),
                                        batch_size = 64,
                                        class_mode = 'binary',
                                        shuffle=False)

score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

Found 688 images belonging to 2 classes.
Test Loss 0.35772812366485596
Test accuracy 0.875


In [17]:
test_set.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

yhat_classes = classifier.predict_classes(test_set, verbose=0)
yhat_classes = yhat_classes[:, 0]

print(classification_report(test_set.classes,yhat_classes))

C:\Users\bruno\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.90      0.85      0.87       344
           1       0.86      0.90      0.88       344

    accuracy                           0.88       688
   macro avg       0.88      0.88      0.87       688
weighted avg       0.88      0.88      0.87       688

